In [1]:
from huggingface_hub import login

In [2]:
login()

In [3]:
import lm_eval
from lm_eval.evaluator_utils import get_task_list, get_sample_size
from lm_eval.tasks.__init__ import get_task_dict

In [4]:
task_dict = get_task_dict(['dharma2'])
task_dict

2024-04-24:19:40:34,739 WARNING  [task.py:322] [Task: dharma2] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
2024-04-24:19:40:34,740 WARNING  [task.py:322] [Task: dharma2] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.


{'dharma2': ConfigurableTask(task_name=dharma2,group_name=None,output_type=generate_until,num_fewshot=None,num_samples=300)}

In [5]:
import argparse
args = argparse.Namespace(
    model='hf',
    model_args='pretrained=meta-llama/Meta-Llama-3-8B',
    tasks='dharma2',
    device='cuda:0',
    batch_size='auto',
    # set limit to None for full test
    limit=2, # 
    # defaults 
    num_fewshot=None,
    max_batch_size=None,
    output_path=None,
    use_cache=None,
    cache_requests=None,
    check_integrity=False,
    write_out=False,
    log_samples=False,
    show_config=False,
    include_path=None,
    gen_kwargs=None,
    verbosity='INFO',
    wandb_args='',
    predict_only=False,
    seed=[0, 1234, 1234],  # Adjust if needed
    trust_remote_code=False,
    # default function params
    rewrite_requests_cache = False,
)

In [6]:
from collections import defaultdict

# tracks all Instances/requests a model must generate output on.
requests = defaultdict(list)

# stores the amount to pad out reqs per req. type so that
# number of fwd passes per distributed rank is equal
padding_requests = defaultdict(int)

In [7]:
task_hierarchy, eval_tasks = get_task_list(task_dict)
print(task_hierarchy)
print(eval_tasks)

defaultdict(<class 'list'>, {'dharma2': []})
[TaskOutput(task_name=dharma2, group_name=None, version=1.0,n_shot=Nonetask_alias=None, group_alias=None)]


In [8]:
if not args.log_samples:
    if not all(
        "bypass" not in getattr(task_output.task, "_metric_fn_list", {}).keys()
        for task_output in eval_tasks
    ):
        raise ValueError("log_samples must be True for 'bypass' metric-only tasks")

In [9]:
lm = lm_eval.api.registry.get_model(args.model).create_from_arg_string(
    args.model_args,
    {
        "batch_size": args.batch_size,
        "max_batch_size": args.max_batch_size,
        "device": args.device,
    },
)

2024-04-24:19:41:00,348 INFO     [huggingface.py:164] Using device 'cuda:0'


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
#for task_output in eval_tasks:
task_output = eval_tasks[0]

In [11]:
task = task_output.task
print(task)
limit = get_sample_size(task, args.limit)
print(limit)

ConfigurableTask(task_name=dharma2,group_name=None,output_type=generate_until,num_fewshot=None,num_samples=300)
2


In [12]:
task.build_all_requests(
    limit=limit,
    rank=lm.rank,
    world_size=lm.world_size,
    cache_requests=args.cache_requests,
    rewrite_requests_cache=args.rewrite_requests_cache,
)

2024-04-24:19:41:18,752 INFO     [task.py:395] Building contexts for dharma2 on rank 0...
100%|██████████| 2/2 [00:00<00:00, 18477.11it/s]


In [13]:
# aggregate Instances by LM method requested to get output.
for instance in task.instances:
    reqtype = instance.request_type
    requests[reqtype].append(instance)

assert lm.world_size == 1
# if lm.world_size > 1: # if is never entered

In [14]:
len(requests['generate_until'])

2

In [15]:
### Run LM on inputs, get all outputs ###
# execute each type of request
for reqtype, reqs in requests.items():
    # create `K` copies of each request `req` based off `K = req.repeats`
    cloned_reqs = []
    for req in reqs:
        cloned_reqs.extend([req] * req.repeats)

    if (lm.world_size > 1) and (padding_requests[reqtype] > 0):
        for _ in range(padding_requests[reqtype]):
            cloned_reqs.extend([req] * req.repeats)

    # run requests through model
    resps = getattr(lm, reqtype)(cloned_reqs)

    # put responses from model into a list of length K for each request.
    for x, req in zip(resps, cloned_reqs):
        req.resps.append(x)

    if lm.world_size > 1:
        lm.accelerator.wait_for_everyone()

Running generate_until requests:   0%|          | 0/2 [00:00<?, ?it/s]

Passed argument batch_size = auto. Detecting largest batch size
Determined Largest batch size: 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Running generate_until requests: 100%|██████████| 2/2 [00:11<00:00,  5.96s/it]


In [16]:
reqs[0]

Instance(request_type='generate_until', doc={'input': 'Question: In areas where limestone is present underneath the soil, continued rains can alter the limestone formation. The limestone can change to a highly porous formation that can lead to the formation of caves and sinkholes. Which process is directly responsible for changing the limestone?\nChoices:\nA: compaction\nB: erosion\nC: faulting\nD: weathering\nAnswer:', 'output': 'D', 'subject': 'ARC-Easy'}, arguments=('Question: In areas where limestone is present underneath the soil, continued rains can alter the limestone formation. The limestone can change to a highly porous formation that can lead to the formation of caves and sinkholes. Which process is directly responsible for changing the limestone?\nChoices:\nA: compaction\nB: erosion\nC: faulting\nD: weathering\nAnswer:', {'until': ['\n\n'], 'do_sample': False}), idx=0, metadata=('dharma2', 0, 1), resps=[' D'], filtered_resps={}, task_name='dharma2', doc_id=0, repeats=1)

In [17]:
reqs[0].doc

{'input': 'Question: In areas where limestone is present underneath the soil, continued rains can alter the limestone formation. The limestone can change to a highly porous formation that can lead to the formation of caves and sinkholes. Which process is directly responsible for changing the limestone?\nChoices:\nA: compaction\nB: erosion\nC: faulting\nD: weathering\nAnswer:',
 'output': 'D',
 'subject': 'ARC-Easy'}

In [18]:
reqs[0].arguments

('Question: In areas where limestone is present underneath the soil, continued rains can alter the limestone formation. The limestone can change to a highly porous formation that can lead to the formation of caves and sinkholes. Which process is directly responsible for changing the limestone?\nChoices:\nA: compaction\nB: erosion\nC: faulting\nD: weathering\nAnswer:',
 {'until': ['\n\n'], 'do_sample': False})

In [21]:
reqs[0].resps[0], reqs[0].doc['output'], reqs[1].resps[0], reqs[1].doc['output']

(' D', 'D', ' B', 'B')

In [22]:
RANK = lm.rank
WORLD_SIZE = lm.world_size

RANK, WORLD_SIZE

(0, 1)

In [23]:
### Postprocess outputs ###

#for task_output in eval_tasks:
task_output = eval_tasks[0]

task = task_output.task
task.apply_filters()

In [24]:
task._filters

[FilterEnsemble(name='remove_whitespace', filters=[functools.partial(<class 'lm_eval.filters.extraction.WhitespaceFilter'>), functools.partial(<class 'lm_eval.filters.selection.TakeFirstFilter'>)])]

In [25]:
### Collect values of metrics on all datapoints ###
# # unpack results and sort back in order and return control to Task
# TODO: make it possible to use a different metric per filter
# Pre-process task.instances to group by doc_id
instances_by_doc_id = defaultdict(list)
for instance in task.instances:
    instances_by_doc_id[instance.doc_id].append(instance)

In [26]:
# Sort instances within each group
for instances in instances_by_doc_id.values():
    instances.sort(key=lambda x: x.idx)

In [27]:
for k in task.instances[0].filtered_resps.keys(): print(k)

remove_whitespace


In [28]:
req.filtered_resps

{'remove_whitespace': 'B'}

In [30]:
# iterate over different filters used
for filter_key in task.instances[0].filtered_resps.keys():
    doc_iterator = task.doc_iterator(rank=RANK, limit=limit, world_size=WORLD_SIZE)
    
    for doc_id, doc in doc_iterator:
        requests = instances_by_doc_id[doc_id]
        metrics = task.process_results(
            doc, [req.filtered_resps[filter_key] for req in requests]
        )
        if args.log_samples:
            target = task.doc_to_target(doc)
            example = {
                "doc_id": doc_id,
                "doc": doc,
                "target": target,
                "arguments": [req.args for req in requests],
                "resps": [req.resps for req in requests],
                "filtered_resps": [
                    req.filtered_resps[filter_key] for req in requests
                ],
            }
            example.update(metrics)
            task_output.logged_samples.append(example)
        for metric, value in metrics.items():
            task_output.sample_metrics[(metric, filter_key)].append(value)